<h1 align='center'>Classificação de spam Machine Learning | Implantação End-to-End | classificador de spam usando Python</h1>

Data Scientist jr: Karina Gonçalves Soares

Links de estudo:

* [Repositório_Spam_classification](https://github.com/EddyGiusepe/Spam_classifier_using_Python/blob/main/spam_classification_ML_End_to_End_Deployment.ipynb)
* [Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)


Nesse script vamos aprender como construir e implementar um modelo de classificação de spam de ponta a ponta (end-to-end) usando NLP Machine Learning. Vamos estudar e visualizar como criar uma NUVEM DE PALAVRAS.

Também aprenderemos como criar e usar o arquivo Pickle do modelo de aprendizado de máquina.

### Importando Bibliotecas

In [2]:
import pandas as pd # Análise de Dados
import numpy as np  # Computação científica e funções matemáticas
import pickle # Converte objetos Python em uma série em Bytes
import nltk # Processamento de Linguagem natural
import re # Fornece recursos para trabalhar com expressões regulares,
          # que são padrões de busca usados para encontrar correspondências em string de texto.
import csv # Manipulação de arquivos csv

import matplotlib.pyplot as plt # Visualização
%matplotlib inline  
import seaborn as sns # Visualização

from sklearn.metrics import accuracy_score, fbeta_score, classification_report # Métricas de avaliação de modelos
from wordcloud import wordcloud # Gera nuvem de palavras a partir de um texto
from nltk.tokenize import word_tokenize

# Divide o texto em tokens, como palavras individuais
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
ss = SnowballStemmer('english')
ps = PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eddygiusepe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Carregando nossos Dados

In [12]:
import csv

with open('spam.csv', mode='r', encoding='ISO-8859-1') as arquivo:
    arquivo_csv = csv.reader(arquivo, delimiter=',')
    linhas = 0
    for coluna in arquivo_csv:
        if linhas == 0:
            print(f'Colunas:  {" ".join(coluna)}')
            linhas += 1
        else:
            print(f'\t{coluna[0]} é o {coluna[1]}')
            linhas += 1 

    print(f'lidas {linhas} linhas.')

        # Faça algo com cada linha do arquivo CSV
        #linhas += 1


Colunas:  v1 v2   
	ham é o Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
	ham é o Ok lar... Joking wif u oni...
	spam é o Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
	ham é o U dun say so early hor... U c already then say...
	ham é o Nah I don't think he goes to usf, he lives around here though
	spam é o FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
	ham é o Even my brother is not like to speak with me. They treat me like aids patent.
	ham é o As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
	spam é o WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To cla

In [14]:

# Ver vídeo para ler um arquivo .csv --> https://www.youtube.com/watch?v=K1RLuCp_LvI

df = pd.read_csv('spam.csv', sep=',', encoding='ISO-8859-1', header=0, usecols=['class', 'menssage'])

df.head()


ValueError: Usecols do not match columns, columns expected but not found: ['menssage', 'class']

In [15]:
msg_df = pd.read_csv('/content/drive/MyDrive/3_EDDY_ISH_TECNOLOGIA/4_ML_inside_of _MANTIS/Spam_classification_ML/spam.csv',
                     sep=',', encoding='ISO-8859-1', header=0, usecols=['class', 'message'])

msg_df.head(5)
#display(msg_df)
     

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/3_EDDY_ISH_TECNOLOGIA/4_ML_inside_of _MANTIS/Spam_classification_ML/spam.csv'